# Note

* [Remark] for remark
* [TBD] for continue

* 2023-06-11 : add get_files_with_specific_route_date, get_xy
* 2023-07-01 : get_date_lst

# Set up

In [22]:
import os
import pandas as pd
from datetime import datetime
import csv #write serach result to csv
from datetime import datetime, date

In [2]:
folder = 'C:\\Users\\Pleng Weerada\\Documents\\MSc Project\\flight_data\\'

# Function

In [3]:
if_print = False

## get_file_name_d0

In [4]:
def get_file_name_d0(route, depart_dt):
    # route, depart_dt = 'LON-to-BKK', ' 1/5'
    if if_print:
        print('-'*30)
        print('get_file_name_d0 \nroute, depart_dt', route, depart_dt)
    
    depart_dt = depart_dt.replace("/","-").replace(" ","")

    des = route.split("-")[2]
    # folder_search = folder + 'raw_files_BKK_d0'
    folder_search = folder + 'raw_files_' + des + '_d0'    
    file_lst = os.listdir(folder_search)
    
    ret_file_nm = ''
    for file in file_lst:
        f = file.split('_')
        # find file that has f[1](depart date) = specific depart_dt
        if f[1] == depart_dt:
            ret_file_nm = file
            break
    return ret_file_nm

In [5]:
get_file_name_d0('LON-to-BKK', ' 1/5')

'LON-to-BKK_1-5_2023-05-01_06-46-04.csv'

## get_price_from_file

In [6]:
def get_price_from_file(file_name, route, depart_dt, airline_nm):
    if if_print:
        print('-'*30)
        print('get_price_from_file')
        print('file_name, route, depart_dt, airline_nm ', file_name, route, depart_dt, airline_nm)
    
    # convert file to dataframe sort by 'direct', 'airline_nm', 'ticket_price' and deduplicate row
    file_name_dir = folder + 'raw_files_BKK_d0\\' + file_name
    df = pd.read_csv(file_name_dir)
    df = df.sort_values(['direct', 'airline_nm', 'ticket_price'],ascending = [False,True, True])
    df = df.drop_duplicates(keep='first')
    
    # search price of specific depart date and airline
    search_df = df.loc[(df['depart_dt'] == depart_dt) & (df['airline_nm'] == airline_nm)]
    if if_print:
        print('search_df ', search_df)
    
    ticket_price = -9999
    if len(search_df) > 0:
        # if price on depart date > 2 then get the lowest price
        search_lst = search_df.head(1).values.tolist()
        ticket_price = float(search_lst[0][3][1:])
        if if_print:
            print('search_lst: \n', search_lst)
            print('ticket_price: \n', ticket_price)
    
    return ticket_price, df

In [7]:
price,df = get_price_from_file('LON-to-BKK_1-5_2023-05-01_06-46-04.csv', 'LON-to-BKK', ' 1/5', 'Thai Airways')
price

526.0

In [8]:
price,df = get_price_from_file('LON-to-BKK_10-5_2023-05-10_08-06-10.csv', 'LON-to-BKK', ' 10/5', 'Thai Airways')
price

471.0

In [9]:
df

route depart_dt            search_dt ticket_price  \
11  LON to BKK      10/5  2023-05-10 08:06:10         £567   
1   LON to BKK      10/5  2023-05-10 08:06:10         £471   
3   LON to BKK      10/5  2023-05-10 08:06:10         £473   
14  LON to BKK      10/5  2023-05-10 08:06:10         £687   
8   LON to BKK      10/5  2023-05-10 08:06:10         £540   
10  LON to BKK      10/5  2023-05-10 08:06:10         £545   
2   LON to BKK      10/5  2023-05-10 08:06:10         £314   
12  LON to BKK      10/5  2023-05-10 08:06:10         £585   
13  LON to BKK      10/5  2023-05-10 08:06:10         £624   
4   LON to BKK      10/5  2023-05-10 08:06:10         £433   
5   LON to BKK      10/5  2023-05-10 08:06:10         £468   
0   LON to BKK      10/5  2023-05-10 08:06:10         £652   
7   LON to BKK      10/5  2023-05-10 08:06:10         £526   
6   LON to BKK      10/5  2023-05-10 08:06:10         £494   
16  LON to BKK      10/5  2023-05-10 08:06:10         £542   

            airline_nm  direct  
11             EVA Air  direct  
1         Thai Airways  direct  
3         Thai Airways  direct  
14   Austrian Airlines  1 stop  
8       Cathay Pacific  1 stop  
10      Cathay Pacific  1 stop  
2       China Southern  1 stop  
12            Emirates  1 stop  
13            Emirates  1 stop  
4       Etihad Airways  1 stop  
5       Etihad Airways  1 stop  
0                  KLM  1 stop  
7       Kuwait Airways  1 stop  
6      Royal Jordanian  1 stop  
16  Vistara, Air India  1 stop

## get_data_from_file

In [10]:
def get_data_from_file(file_name, s_direct, s_airline, data):
    
    print('file_name ', file_name)
    
    ret_data = [] # return data from csv files 
    df_depart_date = pd.DataFrame()  # return data for d0 file
    
    if len(data) > 0:
        ret_data = data

    # get data from csv to dataframe, sort row by 'direct', 'airline_nm', 'ticket_price' and deduplicate row
    df = pd.read_csv(file_name)
    df = df.sort_values(['direct', 'airline_nm', 'ticket_price'],ascending = [False,True, True])
    df = df.drop_duplicates(keep='first') # drop duplicate row by all column
    
    # filter data by direct type, airline name
    if len(s_direct) > 0 and len(s_airline) > 0:
        df = df.loc[(df['direct'] == s_direct) & (df['airline_nm'] == s_airline)]
        df = df.drop_duplicates(subset=['direct', 'airline_nm'], keep='first') # drop duplicate row by direct, airline
    else:
        if len(s_direct) > 0:
            df = df.loc[(df['direct'] == s_direct)]
            df = df.drop_duplicates(subset='direct', keep='first') # drop duplicate row by direct
        if len(s_airline) > 0:
            df = df.loc[(df['airline_nm'] == s_airline)]
            df = df.drop_duplicates(subset='airline_nm', keep='first') # drop duplicate row by airline
    
    # get data
    for index, row in df.iterrows():
        route = row['route'].replace(" ", "-")   
        depart_dt = row['depart_dt']  
        search_dt = row['search_dt'][:10] #get only date, excludes time 
        ticket_price = float(row['ticket_price'][1:])
        airline_nm = row['airline_nm']
        direct = row['direct']
        
        # col_pre_depart_dt
        depart_dt_date = datetime.strptime(depart_dt.replace(" ","")+'/23','%d/%m/%y')
        col_pre_depart_dt = depart_dt_date.strftime("%Y-%m-%d")

        # col_days_bf_depart
        search_dt_date = datetime.strptime(search_dt,'%Y-%m-%d')
        days_diff =  search_dt_date - depart_dt_date
        col_days_bf_depart = days_diff.days

        # col_saving_rate
        depart_file_name = get_file_name_d0(route, depart_dt.replace(" ","")) #'LON-to-BKK_1-5_2023-05-01_06-46-04.csv'
        
        if if_print:
            print('-'*30)
            print('depart_file_name ', depart_file_name)
            
        price_on_depart_date, df_depart_date =  get_price_from_file(depart_file_name, route, depart_dt, airline_nm)
        
        if price_on_depart_date == -9999:
            # not found price on this specific departure date
            col_saving_rate = -9999
            row = [] # not found price on this specific date
        else:
            col_saving_rate = (price_on_depart_date - ticket_price)*(100/price_on_depart_date)
            # create data row
            row = [route, airline_nm, direct, depart_dt, col_pre_depart_dt
               , search_dt, col_days_bf_depart, ticket_price, price_on_depart_date, col_saving_rate ]
            ret_data.append(row)
        
        if if_print:
            print('price_on_depart_date ', price_on_depart_date)
            print('col_saving_rate ', col_saving_rate)
            print('row ', row)

    return ret_data, df_depart_date

In [11]:
file_name = folder + 'raw_files_BKK_dx\\' + 'LON-to-BKK_1-5_2023-04-01_11-10-59.csv'
s_direct = 'direct'
s_airline = 'Thai Airways' #''

ret_data,df = get_data_from_file(file_name, s_direct, s_airline, [])
ret_data[0][6], ret_data[0][9]

file_name  C:\Users\Pleng Weerada\Documents\MSc Project\flight_data\raw_files_BKK_dx\LON-to-BKK_1-5_2023-04-01_11-10-59.csv


(-30, 13.688212927756654)

In [12]:
file_name = folder + 'raw_files_BKK_dx\\' + 'LON-to-BKK_10-5_2023-04-10_15-56-04.csv'
s_direct = 'direct'
s_airline = 'Thai Airways' #''

ret_data,df = get_data_from_file(file_name, s_direct, s_airline, [])
ret_data

file_name  C:\Users\Pleng Weerada\Documents\MSc Project\flight_data\raw_files_BKK_dx\LON-to-BKK_10-5_2023-04-10_15-56-04.csv


[['LON-to-BKK',
  'Thai Airways',
  'direct',
  ' 10/5',
  '2023-05-10',
  '2023-04-10',
  -30,
  463.0,
  471.0,
  1.6985138004246285]]

## get_files_with_specific_route_date

In [13]:
def get_files_with_specific_route_date(s_route, s_depart_dt):
    
    # variables
    specific_lst = [] # list contain file names for specific route, date
    specific_route = s_route         # string: 'BKK'
    specific_depart_dt = s_depart_dt # list, if empty list meaning get all dates
    # print('specific_route, specific_depart_dt ', specific_route, specific_depart_dt)
    
     # get all file with specific route
    fd = folder + 'raw_files_'+ specific_route + '_dx\\'
    file_lst = os.listdir(fd)
    # print('num of files in specific_route ', len(file_lst))
    
    # get all file with specific depart date
    # print('len(specific_depart_dt) ', len(specific_depart_dt))
    for file in file_lst:
        f = file.split("_")
        if len(specific_depart_dt) > 0:
            if f[1] in specific_depart_dt:
                specific_lst.append(file)
        else:
            specific_lst.append(file)
    
    print('num of files in specific_date ', len(specific_lst))
    
    return specific_lst

In [14]:
lst = get_files_with_specific_route_date('BKK', [])
len(lst)

num of files in specific_date  1031


1031

In [15]:
lst = get_files_with_specific_route_date('BKK', ['1-5'])
len(lst), lst[0]

num of files in specific_date  30


(30, 'LON-to-BKK_1-5_2023-04-01_11-10-59.csv')

## get_xy

In [16]:
def get_xy(s_lst, s_route, s_direct, s_airline):
    
    specific_lst = s_lst         # list of specific files to convert to x, y
    specific_route = s_route
    specific_direct = s_direct   # 'direct'
    specific_airline = s_airline # 'Thai Airways'
    
    x = [] # days before depart date
    y = [] # saving rate
    ret_lst = [] # list of raw data with all columns from files
    
    count_files = 0
    count_xy = 0
    start_time = datetime.now()
    
    for file in specific_lst:
        # print(file)
        file_name = folder + 'raw_files_'+ specific_route + '_dx\\' + file

        lst, df = get_data_from_file(file_name, specific_direct, specific_airline, [])
        count_files += 1
        
        if if_print:
            print('ret_lst ', ret_lst)

        if len(lst) > 0:
            ret_lst.append(lst)
            x.append([lst[0][6]])   # days before depart
            y.append(lst[0][9])   # saving rate
            count_xy += 1

    end_time = datetime.now()
    
    # [TBD] create csv file
    preprocess_file_name = folder + 'preprocess_files\\' + 'ticket_price_' + s_route + '_' \
                           + s_direct + '_' + s_airline +'_' + datetime.now().strftime("%Y-%m-%d_%H-%M-%S") + '.csv'
    header = ['route','airline_nm','direct','depart_dt','col_pre_depart_dt','search_dt'
              ,'col_days_bf_depart','ticket_price','price_on_depart_date','col_saving_rate']
    with open(preprocess_file_name, 'w', encoding='UTF8', newline='') as f:
        writer = csv.writer(f)

        # write the header
        writer.writerow(header)

        # write multiple rows
        writer.writerows(ret_lst)
    
    print("-"*30)
    print('pre process file name ' , preprocess_file_name)
    print("Finish!! processing time {0}-{1} \nnumber of processed files: {2} \nlen x,y: {3}".format(
        start_time.strftime("%H:%M:%S"), end_time.strftime("%H:%M:%S"), count_files, count_xy))

    return x, y, ret_lst, df

In [17]:
x, y, ret_lst, df = get_xy([ 'LON-to-BKK_1-5_2023-04-01_11-10-59.csv'
                           , 'LON-to-BKK_10-5_2023-05-04_06-52-19.csv'
                           , 'LON-to-BKK_10-5_2023-05-03_06-30-28.csv']
                           , 'BKK', 'direct', 'Thai Airways')

file_name  C:\Users\Pleng Weerada\Documents\MSc Project\flight_data\raw_files_BKK_dx\LON-to-BKK_1-5_2023-04-01_11-10-59.csv
file_name  C:\Users\Pleng Weerada\Documents\MSc Project\flight_data\raw_files_BKK_dx\LON-to-BKK_10-5_2023-05-04_06-52-19.csv
file_name  C:\Users\Pleng Weerada\Documents\MSc Project\flight_data\raw_files_BKK_dx\LON-to-BKK_10-5_2023-05-03_06-30-28.csv
------------------------------
pre process file name  C:\Users\Pleng Weerada\Documents\MSc Project\flight_data\preprocess_files\ticket_price_BKK_direct_Thai Airways_2023-07-01_17-24-26.csv
Finish!! processing time 17:24:26-17:24:26 
number of processed files: 3 
len x,y: 2


In [18]:
# expected: ([[-30]], [13.688212927756654], 1036, 12)
x, y, len(ret_lst), len(df)

([[-30], [-7]], [13.688212927756654, -0.42462845010615713], 2, 15)

In [19]:
ret_lst

[[['LON-to-BKK',
   'Thai Airways',
   'direct',
   ' 1/5',
   '2023-05-01',
   '2023-04-01',
   -30,
   454.0,
   526.0,
   13.688212927756654]],
 [['LON-to-BKK',
   'Thai Airways',
   'direct',
   ' 10/5',
   '2023-05-10',
   '2023-05-03',
   -7,
   473.0,
   471.0,
   -0.42462845010615713]]]

## get_date_lst

In [26]:
def get_date_lst (route, date_range_lst):
    dates_lst_train = []
    for d in dates_train:
        str_dt = d.strftime('%d-%m')
        format_dt = ''
        if str_dt[0] == '0':
            format_dt = str_dt[1:2]+'-'+str_dt[4:]
        else:
            format_dt = str_dt[:2]+'-'+str_dt[4:]
        dates_lst_train.append(format_dt)
    len(dates_lst_train), dates_lst_train[0]

    lst_train = get_files_with_specific_route_date(route, dates_lst_train)
    return lst_train

In [27]:
dates_train = pd.date_range(date(2023, 4, 24),date(2023, 4, 28))
dates_train

DatetimeIndex(['2023-04-24', '2023-04-25', '2023-04-26', '2023-04-27',
               '2023-04-28'],
              dtype='datetime64[ns]', freq='D')

In [28]:
get_date_lst ('BKK', dates_train)

num of files in specific_date  150


['LON-to-BKK_24-4_2023-03-25_10-29-03.csv',
 'LON-to-BKK_24-4_2023-03-26_20-06-10.csv',
 'LON-to-BKK_24-4_2023-03-27_23-35-43.csv',
 'LON-to-BKK_24-4_2023-03-28_11-44-42.csv',
 'LON-to-BKK_24-4_2023-03-29_08-46-07.csv',
 'LON-to-BKK_24-4_2023-03-30_19-48-24.csv',
 'LON-to-BKK_24-4_2023-03-31_20-12-43.csv',
 'LON-to-BKK_24-4_2023-04-01_10-25-01.csv',
 'LON-to-BKK_24-4_2023-04-02_14-09-40.csv',
 'LON-to-BKK_24-4_2023-04-03_17-47-45.csv',
 'LON-to-BKK_24-4_2023-04-04_11-09-59.csv',
 'LON-to-BKK_24-4_2023-04-05_19-39-18.csv',
 'LON-to-BKK_24-4_2023-04-06_21-21-14.csv',
 'LON-to-BKK_24-4_2023-04-07_05-44-11.csv',
 'LON-to-BKK_24-4_2023-04-08_10-41-42.csv',
 'LON-to-BKK_24-4_2023-04-09_11-28-56.csv',
 'LON-to-BKK_24-4_2023-04-10_13-29-19.csv',
 'LON-to-BKK_24-4_2023-04-11_09-18-39.csv',
 'LON-to-BKK_24-4_2023-04-12_09-14-42.csv',
 'LON-to-BKK_24-4_2023-04-13_09-25-42.csv',
 'LON-to-BKK_24-4_2023-04-14_13-59-28.csv',
 'LON-to-BKK_24-4_2023-04-15_09-32-59.csv',
 'LON-to-BKK_24-4_2023-04-16_10-